<a href="https://colab.research.google.com/github/MLo7Ghinsan/DiffSinger_colab_notebook_MLo7/blob/main/g2p_for_OpenUtau_training_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is a notebook for training G2P files for OpenUtau.

The original code can be found [here](https://github.com/stakira/OpenUtau/tree/master/py).

This notebook is an edited copy of Lotte (how do i even tag you here help)

#Setup

In [1]:
#@title # Mount Google Drive and Setup

from google.colab import drive
from IPython.display import clear_output

!rm -rf /content/sample_data
drive.mount("/content/drive")
!git clone https://github.com/stakira/OpenUtau.git
#thank you lotte <3
!pip install antlr4-python3-runtime==4.9.*
!pip install hydra-core==1.3.2 omegaconf==2.3.0 -q #that warning popup is annoying
!pip install torch==2.1.0 torchaudio==2.1.0
!pip install editdistance==0.6.2
!pip install tqdm==4.65.0
!pip install onnx
!pip install PyYAML
#moving it cus like i dont think we need the whole ou folder
!mv /content/OpenUtau/py/g2p /content
!rm -rf OpenUtau
%cd /content/g2p

clear_output()

# Training

In [ ]:
#@title Input dictionary
from IPython.display import clear_output
%cd /content/g2p
import os
if not os.path.exists("/content/user_g2p_data"):
    os.makedirs("/content/user_g2p_data")
clear_output()
import sys
import re
import torch
import hydra
import yaml
from omegaconf import OmegaConf
sys.path.append(os.path.abspath('.'))
from dataset import SphinxDataset
from trainer import G2pTrainer
from model import GreedyG2p

def train(trainer):
    print('training...')
    trainer.train()

def export(trainer, model_path, onnx_path):
    print('exporting model...')
    trainer.model.load_state_dict(torch.load(model_path))
    trainer.model.cpu()
    greedy = GreedyG2p(trainer.model.max_len,
                       trainer.model.encoder, trainer.model.decoder)
    greedy.export(onnx_path)

    print('testing...')
    trainer.test('test_log.txt')

#@markdown input dictionary (can be dict.txt or dsdict.yaml)
dict_path = "" # @param {type:"string"}

#@markdown input model save location
save_path = "" # @param {type:"string"}
if not save_path:
    raise ValueError("Empty save_path, please specify a path to save your model")
else:
    pass

user_dict_path = save_path +"/user_dictionary.txt" #cleaned dict
user_config = save_path + "/config.yaml" #generated config
user_phonemes = save_path + "/phones.txt" #yea phoneme list yea poosay

default_config = {
    "_target_": "model.G2p",
    "max_len": 48,
    "encoder": {
        "_target_": "model.Encoder",
        "graphemes": [],
        "d_model": 64,
        "d_hidden": 128,
        "num_layers": 2,
        "dropout": 0.1
    },
    "decoder": {
        "_target_": "model.Decoder",
        "phonemes": [],
        "d_model": 64,
        "d_hidden": 128,
        "num_layers": 2,
        "dropout": 0.1
    }
}

with open(user_config, "w") as cfg:
    yaml.dump(default_config, cfg)

if dict_path:
    pass
else:
    raise ValueError("Please input path to your dictionary")

if dict_path.endswith(".txt"):
    print("using txt format")
    with open(dict_path, "r", encoding = "utf-8") as dict:
        content = dict.read()
        if "\t" in content:
            no_tab_content = content.replace("\t", "  ")
            with open(user_dict_path, "w", encoding = "utf-8") as file:
                file.write(no_tab_content)
        else:
            with open(user_dict_path, "w", encoding = "utf-8") as file:
                file.write(content)

elif dict_path.endswith(".yaml"):
    print("using yaml format")
    #because they're troublesome >:( (not sure about true and false tho, but ill just include them cus why not)
    bool2string = {
        "yes": "'yes'",
        "no": "'no'",
        "on": "'on'",
        "off": "'off'",
        "true": "'true'",
        "false": "'false'"
    }

    #ik its extra but oh well
    with open(dict_path, "r", encoding = "utf-8") as file:
        content = file.read()
        for word, replacement in bool2string.items():
            pattern = r"(?<!\')\b{}\b(?!\')".format(re.escape(word))
            content = re.sub(pattern, replacement, content)
    with open(dict_path, "w", encoding = "utf-8") as file:
        file.write(content)

    with open(dict_path, "r", encoding = "utf-8") as dict:
        content = yaml.safe_load(dict)
        entries = content.get("entries", [])
        with open(user_dict_path, "w", encoding = "utf-8") as file:
            for entry in entries:
                grapheme = entry["grapheme"]
                phonemes = " ".join(entry["phonemes"])
                file.write(f"{grapheme}  {phonemes}\n")
else:
    raise TypeError("format not supported")

graphemes = set()
phonemes = set()

with open(user_dict_path, "r", encoding = "utf-8") as file:
    for line in file:
        line = line.strip()
        if line:
            grapheme = line.split()[0]
            graphemes.add(grapheme)
            phoneme = line.split("  ")[1]
            phonemes.update(set(phoneme.split()))

#add the necessary stuff ig based on readme

required_char = ["<unk>", "<pad>", "<bos>", "<eos>"]

#edited this for when theres like special character in dict'''\\
for char in required_char:
    graphemes.discard(char)
    phonemes.discard(char)

graphemes = required_char + sorted(graphemes)
phonemes = required_char + sorted(phonemes)

vowel_types = {"a", "i", "u", "e", "o", "N", "M", "NG"}
with open(user_phonemes, "w") as f:
    for phoneme in phonemes:
        if phoneme in required_char:
            continue
        if phoneme in vowel_types:
            f.write(f"{phoneme}\tvowel\n")
        else:
            f.write(f"{phoneme}\t-\n")

with open(user_config, "r", encoding = "utf-8") as cfg:
    training_config = yaml.safe_load(cfg)
training_config["encoder"]["graphemes"] = graphemes
training_config["decoder"]["phonemes"] = phonemes
with open(user_config, "w", encoding = "utf-8") as cfg:
    yaml.dump(training_config, cfg, allow_unicode = True)

cfg = user_config
cfg = OmegaConf.load(cfg)

dataset = user_dict_path
dataset = SphinxDataset(dataset, cfg,
                        comment_prefix=';;;',
                        # "RECORDS(1)" -> "RECORDS"
                        remove_word_digits=True,
                        # "R EH1 K ER0 D Z" -> "R EH K ER D Z"
                        remove_phoneme_digits=True)

# @markdown you may need to adjust the batch size and epochs. <br> If there's too much loss, you can try decreasing the batch size. <br> Between 50 and 150 epochs is generally recommended for training, although you can play around with this a bit. <br> You can continue training from the latest checkpoint at a later moment if you so desire.
loss_device = "cuda" # @param ["cpu", "cuda"]
batch_size = 150 # @param {type:"slider", min:1, max:300, step:1}
epochs = 120 # @param {type:"slider", min:1, max:300, step:1}


#@markdown the model that get saves to save_path will be the g2p-best.ptsd, you can check in g2p folder under /content for other model if applicable

In [ ]:
#@title Start training

#@markdown This option to test the finished model for its word error and phoneme error rate.... very slow though, especially with big model
test_for_error = False # @param {type:"boolean"}

trainer = G2pTrainer(
    device=torch.device("cuda" if torch.cuda.is_available() else "cpu"),
    loss_device=torch.device(loss_device),
    model=hydra.utils.instantiate(cfg),
    dataset=dataset,
    batch_size=batch_size,
    epochs=epochs)

train(trainer)

!cp /content/g2p/g2p-best.ptsd $save_path

if test_for_error:
    trainer.test(test_log = save_path +"/test_log.txt")
else:
    pass

#Export ONNX

In [ ]:
#@title Convert best checkpoint (not here yet lmao if you wanna use it then edit the code in this cell to your path(s), ill come back later to make it look a lil nicer)
model_path = ""
output_path = "" #+filename ill add that later cus im not so sure about the code above yet
export(trainer, model_path, output_path)
